# If there exists only ONE Canadian, what climate does THE Canadian experience?

Canada has quite a bit of land, but the majority of us live compactly in major cities. Suppose the entire Canadian population is projected on to one individual, what climate does this individual experience throughout the year?

To answer this question, I need data on: 
* Canadian population size for different cities
* Daily climate data for different cities

Assumptions: 

Simplification - To simplify the process, I decided to use the top 20 cities out of the top 100 most populated cities in Canada. Which sums up to roughly 80% of the total of the 100 cities.

Data Collection - Since I can't find any clean climate data by city, each city climate data are scraped separately. Notice that the url(s) contain a station ID. I explored historical climate data on the Government of Canada website and grabbed the station IDs with the most reliable data. 

Missing Values - Some days have missing values, which is a problem when I'm trying to perform dot product later on. To solve this problem, the missing data are filled in using interpolation method.

In Complete Data - Since I can't find enough data for the city St. Catharines, it is dropped from the result and replaced by the runner up city in terms of population size

Scraping Function:

The scraping function ScrapeWeather() used in this file was developed in my previous project, where the development in this basic function for a single web-page is explained in detial. 
URL: http://nbviewer.jupyter.org/github/yxtzhu/WeatherScrape/blob/master/Webscrape.ipynb

In [2]:
# Import libraries for webscraping

import urllib.request
from bs4 import BeautifulSoup

# Import libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import datetime
from datetime import datetime

In [186]:
# Function used to scrape a single site from the Government of Canada

def ScrapeWeather(page):
    
    # Query the website
    f=urllib.request.urlopen(page).read()
    
    # Parse the page using beautiful soup
    soup = BeautifulSoup(f, 'html.parser')
    
    # Find the section that contains the year and the month
    option = soup.find_all('div', class_='col-md-6 col-sm-6 col-xs-6 text-center mrgn-tp-md mrgn-bttm-md')
    option=option[0]
    option=option.find_all('option', selected=True)
    
    # Find the table
    table = soup.find_all('table')
    
    # Find title
    th_all = soup.find_all('th')
    result = []
    for th in th_all:
        result.append(th.find('abbr', text=True))

    title=[]
    for abbr in result:
        if abbr is not None:
            title.append(abbr.string)
            
    # clean up title
    for i in range(len(title)):
        if title[i]=='mm':
            title[i]="Total Rain"
        elif title[i]=='cm':
            title[i]="Total Snow"
    
    # Find data
    table_body = soup.find('tbody')
    tr = table_body.find_all('tr')
    td=[]
    date=[]
    data=[]
    for row in tr:
        d=[]
        for r in row.find_all('td'):
            date.append(r.find('abbr'))
            d.append(r.string)
        data.append(d)
    
    # clean date arrays
    t=[]
    for d in date:
        if d is not None:
            t.append(d.get('title'))
    
    date=[]
    for j in t:
        date.append(datetime.strptime(j, '%B %d, %Y'))

    # clean up data array
    for entry in data:
        if entry == []:
            data.remove(entry)

    for entry in data:
        if len(entry)>11:
            entry.pop(0)

    # There should be as many rows as the number dates
    for rm in range(len(data)-len(date)):
        data.pop(len(data)-1)
        rm -=1
    
    for rm in range(len(title)-len(data[0])):
        title.pop(len(title)-1)
        rm-=1
    
    # Construct dataframe for scraped data
    df = pd.DataFrame(data=data,index=date,columns=title)
    df=df.apply(pd.to_numeric, errors="coerce")
    
    # Return the result
    return df

In [180]:
# Get Mean Temp from dataframe
def getMean(df):
    Mmean=df["Mean Temp"]
    return Mmean

In [181]:
# Get Max Temp from dataframe
def getMax(df):
    Mmax=df['Max Temp']
    return Mmax

In [182]:
# Get Min Temp from dataframe
def getMin(df):    
    Mmin=df['Min Temp']
    return Mmin

In [183]:
# Get index from dataframe
def getInex(df):
    Mindex=df.index[0]
    return Mindex

In [ ]:
# Wikipedia has a nice table with Canadian population size by city

In [184]:
# Get population data by Canadian cities for the year of 2016
p='https://en.wikipedia.org/wiki/List_of_the_100_largest_population_centres_in_Canada'
f=urllib.request.urlopen(p).read()

In [185]:
# Parse the page using beautiful soup
soup = BeautifulSoup(f, 'html.parser')
    
# Find the table
table = soup.find_all('table')

In [11]:
# Find table rows
tr = table[0].find_all('tr')

In [187]:
# Going through the table with top 100 cities, and grab city name and population size
td=[]
city=[]
data=[]
for row in tr:
    d=[]
    #print(row.find_all('a').string)
    for r in row.find_all('td'):
        c=row.find('a').string
        if not c in city: 
            city.append(c)
        d.append(r.string)
    data.append(d)

# We should have all 100 cities in the result
print(len(city))

100


In [188]:
# Get headers of the table
header=[]
for h in tr[0].find_all('th'):
    header.append(h.string)
    
# Check headers
print(header)

['Rank', 'Population centre', 'Population in 2016', 'Population in 2011', 'Class']


In [189]:
# Clean up scraped data by removing empty results
for entry in data:
    if entry == []:
        data.remove(entry)

# Make sure we still have 100 entries
print(len(data))

100


In [190]:
# Piece everything together into a dataframe, and drop the columns I don't need
cityframe=pd.DataFrame(data=data,index=city,columns=header)
cityframe=cityframe.drop(['Rank','Class','Population centre', 'Population in 2011'], axis=1)

# Notice that the numerical data are comma delimited strings, remove commas so we can convert it into numbers 
cityframe['Population in 2016'] = cityframe['Population in 2016'].str.replace(',', '')

In [191]:
# Convert data into numeric values and check the total Canadian population size in 2016 for the top 100 cities
cityframe=cityframe.apply(pd.to_numeric, errors="ignore")
totalpop=sum(cityframe['Population in 2016'])
print(totalpop/35151728)

0.696081882518


In [192]:
# Calculate percentage of each entry per table and display the top 20 
cityframe.apply(lambda x: x / float(x.sum())).head(n=20)

,Population in 2016
Toronto,0.221899
Montreal,0.143842
Vancouver,0.092561
Calgary,0.050582
Edmonton,0.043429
Ottawa,0.040446
Winnipeg,0.029096
Quebec City,0.028817
Hamilton,0.028349
Kitchener,0.019209


In [193]:
# What percentage of the top 100 cities does the top 20 cities represent?
cityframe.apply(lambda x: x / float(x.sum())).head(n=20).sum()

Population in 2016    0.806581
dtype: float64

In [194]:
# Build a list of station ids for each of the top 20 cities
# This is manual since I didn't find a list with all the station IDs mapped to the city names
stations=[51459,51157,51442,50430,50149,49568,27174,26892,49908,48569,50093,51337,50620,48649,52838,47707,53000,28011,50089,51117]

In [195]:
# Make sure I got the station ID for all top 20
print(len(stations))

20


In [196]:
# Not enough sufficient daily data on st. Catharines for 2016 can be found
# Let's remove it from the list (st. Catharines,id: 53000)
# Add city ranking number 21 into the list in its place
cityframe.apply(lambda x: x / float(x.sum())).head(n=21)

,Population in 2016
Toronto,0.221899
Montreal,0.143842
Vancouver,0.092561
Calgary,0.050582
Edmonton,0.043429
Ottawa,0.040446
Winnipeg,0.029096
Quebec City,0.028817
Hamilton,0.028349
Kitchener,0.019209


In [197]:
# Add Barrie (ranking number 21) to station IDs and remove St. Catharines
stations.append(42183)
stations.remove(53000)

# Make sure we're back at 20 cities
print(len(stations))

20


In [198]:
# Get the top 21 population % and remove st. Catharines
weight=cityframe.apply(lambda x: x / float(x.sum())).head(n=21)
weight=weight.drop(['St. Catharines'])

# Let's take a look
print(weight)

             Population in 2016
Toronto                0.221899
Montreal               0.143842
Vancouver              0.092561
Calgary                0.050582
Edmonton               0.043429
Ottawa                 0.040446
Winnipeg               0.029096
Quebec City            0.028817
Hamilton               0.028349
Kitchener              0.019209
London                 0.015671
Victoria               0.013720
Halifax                0.012943
Oshawa                 0.012623
Windsor                0.011732
Saskatoon              0.010020
Regina                 0.008772
St. John's             0.007292
Kelowna                0.006210
Barrie                 0.005951


In [199]:
# Now let's build a url list for the year of 2016 for the top 20 cities to be scraped later
pagelist=[]
now = datetime.now()
for id in stations:
    lst=[]
    for month in range(1, 13):
        url='http://climate.weather.gc.ca/climate_data/daily_data_e.html?StationID='+str(id)+'&timeframe=2&StartYear=1840&EndYear=2018&Day=7&Year=2016'+'&Month='+str(month)+'#'
        lst.append(url)
    pagelist.append(lst)
        
# Check to make sure we have an url for each month, and 12 months for the year
print(len(pagelist))
print(len(lst))

20
12


In [226]:
# Let's grab the Mean Temp for now, the scrapeWeather function is modified for it
# n is a counter to help me figure out if and where ScrapeWeather fails
result=[]
results=[]
n=1
for lst in pagelist:
    r=[]
    for page in lst:
        df=ScrapeWeather(page)
        r.append(getMean(df))
    result.append(pd.concat(r))
    print(n)
    n+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [209]:
# Let's get the mean and max ones as well
resultmin=[]
resultmax=[]
resultsmin=[]
resultsmax=[]
n=1
for lst in pagelist:
    rmin=[]
    rmax=[]
    for page in lst:
        df=ScrapeWeather(page)
        rmin.append(getMin(df))
        rmax.append(getMax(df))
    resultmin.append(pd.concat(rmin))
    resultmax.append(pd.concat(rmax))

In [327]:
# Let's remove St. Catharines from the city names
cities=city[0:21]
cities.remove('St. Catharines')
print(len(cities))
print(len(result))

20
20


In [328]:
# Construct dataframes for Mean Temp, Min Temp and Max Temp
results = pd.concat(result, axis=1, join_axes=[result[0].index], keys=cities)
resultsmin = pd.concat(resultmin, axis=1, join_axes=[result[0].index], keys=cities)
resultsmax = pd.concat(resultmax, axis=1, join_axes=[result[0].index], keys=cities)

In [329]:
# Let's print the dataframes and take a quick look
print("Print mean temp")
print(results.head())
print("Print min temp")
print(resultsmin.head())
print("Print max temp")
print(resultsmax.head())

Print mean temp
            Toronto  Montreal  Vancouver  Calgary  Edmonton  Ottawa  Winnipeg  \
2016-01-01     -2.3      -2.0       -1.8     -5.4      -8.2    -2.3      -8.9   
2016-01-02     -2.0      -2.1       -2.0     -9.2     -11.4    -3.9      -7.8   
2016-01-03     -5.0      -7.1       -1.2     -9.5     -14.9    -8.5     -10.9   
2016-01-04    -13.3     -16.5        0.8     -7.5     -15.7   -17.1     -10.1   
2016-01-05     -8.9     -13.8        0.6    -10.5     -16.4   -16.1     -10.7   

            Quebec City  Hamilton  Kitchener  London  Victoria  Halifax  \
2016-01-01         -5.3      -2.6       -3.2    -3.2      -0.2     -2.2   
2016-01-02         -3.2      -1.3       -2.9    -2.3      -0.1     -1.9   
2016-01-03         -6.7      -4.4       -4.8    -4.5       0.6      0.2   
2016-01-04        -15.2     -13.3      -13.6   -12.7       1.2     -6.3   
2016-01-05        -20.0      -9.8      -11.5   -10.6       1.7    -11.4   

            Oshawa  Windsor  Saskatoon  Regina

In [330]:
# Check to see if the dot product makes sense
print(results.dot(weight).head(n=10))

            Population in 2016
2016-01-01           -2.590282
2016-01-02           -2.805845
2016-01-03           -4.897036
2016-01-04                 NaN
2016-01-05                 NaN
2016-01-06           -3.625227
2016-01-07           -2.648498
2016-01-08                 NaN
2016-01-09           -0.650714
2016-01-10           -1.436209


In [233]:
# As shown above, there are many missing values. 
# To work around it, I will fill the missing value with interpolations for the purpose of calculating dot products
results=results.interpolate()
wMeanTemp=results.dot(weight)

# Make sure there's no more missing values
print(results.dot(weight).isnull().sum())

Population in 2016    0
dtype: int64


In [237]:
# Rename the title for the weighted average temp
wMeanTemp.columns=['Weighted Mean Temp']
print(wMeanTemp.head())

            Weighted Mean Temp
2016-01-01           -2.590282
2016-01-02           -2.805845
2016-01-03           -4.897036
2016-01-04           -9.216998
2016-01-05           -8.102497


In [238]:
# Now do the same for the rest
resultsmin=resultsmin.interpolate()
wMinTemp=resultsmin.dot(weight)

resultsmax=resultsmax.interpolate()
wMaxTemp=resultsmax.dot(weight)

wMinTemp.columns=['Weighted Min Temp']
wMaxTemp.columns=['Weighted Max Temp']

# Quick check on the weighted Min Temp and the weighted Max Temp
print(wMinTemp.head())
print(wMaxTemp.head())

            Weighted Min Temp
2016-01-01          -5.033347
2016-01-02          -5.513924
2016-01-03          -9.763833
2016-01-04         -12.108277
2016-01-05         -12.955989
            Weighted Max Temp
2016-01-01          -0.107143
2016-01-02          -0.056998
2016-01-03          -0.020431
2016-01-04          -6.314519
2016-01-05          -3.227112


In [239]:
# Plotting time!

In [240]:
# Import plotly
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [242]:
# Set plotting variables

Maxtrace = go.Bar(
    x=wMaxTemp.index,
    y=wMaxTemp['Weighted Max Temp'],
    name='Weighted Max Temp',
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

Mintrace = go.Bar(
    x=wMinTemp.index,
    y=wMinTemp['Weighted Min Temp'],
    name='Weighted Min Temp',
    marker=dict(
        color='rgb(178,255,102)',
        line=dict(
            color='rgb(76,153,0)',
            width=1.5,
        )
    ),
    opacity=0.6
)

Meantrace = go.Scatter(
    x = wMeanTemp.index,
    y = wMeanTemp['Weighted Mean Temp'],
    name='Weighted Mean Temp',
    line = dict(
        color = ('rgb(255, 175, 102)'),
        width = 3,
        dash = 'dot')
)
data = [Mintrace, Maxtrace, Meantrace]
layout = go.Layout(
    title = 'The 2016 Climate for ONE Canadian'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='OneCA2016')

In [266]:
# Let's find out what portion of the year are below freezing!
freeze=sum(n <= 0 for n in wMinTemp.values.flatten())

print('Out of %d days in year 2016, %d days were less than or equal to the freezing point. \
That\'s %d percent of the time!!' %(len(wMinTemp), freeze, 100*(freeze/len(wMinTemp))))

Out of 366 days in year 2016, 133 days were less than or equal to the freezing point. That's 36 percent of the time!!


In [269]:
# According to the Canadian Food Inspection Agency, refrigerator should be set to 4°C
# CFIA website: http://www.inspection.gc.ca/food/non-federally-registered/safe-food-production/guide/eng/1352824546303/1352824822033
fridge=sum(n <= 4 for n in wMinTemp.values.flatten())

print('Out of %d days in year 2016, %d days were less than or equal to the Canadian Food Inspection Agency \
standard for refrigeration temperature. The Canadian lives %d percent of the time in a fridge!!' \
      %(len(wMinTemp), fridge, 100*(fridge/len(wMinTemp))))

Out of 366 days in year 2016, 206 days were less than or equal to the Canadian Food Inspection Agency standard for refrigeration temperature. The Canadian lives 56 percent of the time in a fridge!!


In [334]:
# Scatterplot max and min temp

Mintrace = go.Scatter(
    x = wMinTemp.index,
    y = wMinTemp['Weighted Min Temp'],
    name='Min Temp',
    mode='markers',
    marker=dict(
        size='16',
        color = np.random.randn(len(wMinTemp)),
        colorscale='Viridis',
        showscale=False
    )
)

Maxtrace = go.Scatter(
    x = wMaxTemp.index,
    y = wMaxTemp['Weighted Max Temp'],
    name='Max Temp',
    mode='markers',
    marker=dict(
        size='16',
        color = np.random.randn(len(wMaxTemp)),
        colorscale='Magma',
        showscale=False
    )
)

d = [Mintrace, Maxtrace]
layout2= go.Layout(
    title= 'Extreme climate of one Canadian',
    hovermode= 'closest')
fig= go.Figure(data=d, layout=layout2)
py.iplot(d, filename='scatterTemp')

In [331]:
# Plot Mean Temp for the top 5 cities, not this is not weighted values

Toronto = go.Scatter(
    x = results.index,
    y = results['Toronto'],
    name='Toronto',
    mode='markers',
    marker=dict(
        size='8',
        color = ('rgb(158,202,225)')
    ),
    opacity=0.6
)

Montreal = go.Scatter(
    x = results.index,
    y = results['Montreal'],
    name='Montreal',
    mode='markers',
    marker=dict(
        size='8',
        color = ('rgb(255, 153, 153)'),
    ),
    opacity=0.6
)

Vancouver = go.Scatter(
    x = results.index,
    y = results['Vancouver'],
    name='Vancouver',
    mode='markers',
    marker=dict(
        size='8',
        color = ('rgb(153, 153, 255)')
    ),
    opacity=0.6
)

Calgary = go.Scatter(
    x = results.index,
    y = results['Calgary'],
    name='Calgary',
    mode='markers',
    marker=dict(
        size='8',
        color = ('rgb(255, 175, 10)')
    ),
    opacity=0.6
)

Edmonton = go.Scatter(
    x = results.index,
    y = results['Edmonton'],
    name='Edmonton',
    mode='markers',
    marker=dict(
        size='8',
        color = ('rgb(204, 255, 153)')
    ),
    opacity=0.6
)
dmean = [Toronto, Montreal, Vancouver, Calgary, Edmonton]

py.iplot(dmean, filename='temp')